### Система Retrieval-Augmented Generation (RAG) с использованием LangChain 🦜🔗

---



## Загружаем все нобходимое



In [ ]:
!pip install langchain faiss-cpu transformers sentence-transformers huggingface_hub langchain_community openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.0/408.0 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.

In [ ]:
!apt-get install git


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
!git clone https://github.com/Gosha69228/TEXT.git


Cloning into 'TEXT'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 57 (delta 20), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (57/57), 50.33 KiB | 831.00 KiB/s, done.
Resolving deltas: 100% (20/20), done.


# Делаем базу данных

---
Прошу обратить внимание на файл TEST_FILE.txt, этот файл сожержит специально выдуманную информацию. Файл может быть использован для проверки того, что модель действительно задействет информацию из базы знаний. (естественно можно вводить вопросы и по теме ИБ). Все статьи по ИБ взяты с официального сайта Касперского и SecurityLab: https://encyclopedia.kaspersky.ru/glossary/insider-threat/ и https://www.securitylab.ru/analytics/

Примеры промтов для файла TEST_FILE.txt:

1) "Как защитить свои данные от злобных единорогов и космических мышей"

2) "Зачем использовать фиолетовые пароли?"

3) "Перед чем не могут устоять единороги"


In [ ]:
import sqlite3

conn = sqlite3.connect('knowledge_base.db')
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS articles (
    id INTEGER PRIMARY KEY,
    title TEXT,
    content TEXT
)
''')
conn.commit()

In [ ]:
import os

repo_path = '/content/TEXT'

for filename in os.listdir(repo_path):
    if filename.endswith('.txt'):
        with open(os.path.join(repo_path, filename), 'r', encoding='utf-8') as file:
            content = file.read()
            title = filename[:-4]
            cursor.execute('INSERT INTO articles (title, content) VALUES (?, ?)', (title, content))

conn.commit()


**Разделяем текст с помощью сплитера на смысловые фрагменты**

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap  = 700,
    length_function = len,
    is_separator_regex = False,
)

In [ ]:
cursor.execute("SELECT id, content FROM articles")
rows = cursor.fetchall()

all_chunks = []

for row in rows:
    article_id, content = row

    texts = text_splitter.create_documents([content])

    all_chunks.extend(texts)

Посмотрим какие фрагменты у нас получились.

В all_chunks[0-208]

In [ ]:
print(all_chunks[0].page_content)

Инсайдерская угроза — риск причинения ущерба организации людьми, которые находятся внутри контура безопасности. К таким людям — их называют инсайдерами — могут относиться как бывшие или действующие сотрудники самой компании, так и специалисты подрядчиков или партнеры, то есть все, кто имеет доступ к конфиденциальной информации и критической инфраструктуре компании.

Инсайдеры могут быть как злонамеренными, так и просто неосторожными. Например, сотрудник, потерявший флешку с секретными данными, — тоже инсайдер. Подробнее о классификации инсайдеров рассказано в «Базе знаний».

Опасность инсайдерской угрозы
Инсайдеры имеют легитимный доступ в компьютерную сеть компании, который нужен им, чтобы выполнять свои обязанности, и средства безопасности не рассматривают их как угрозу. Чтобы получить доступ к данным, им не требуется взламывать чужие аккаунты и обходить средства защиты периметра (межсетевые экраны и антивирусные программы). Используя свое положение, инсайдеры могут изучить действующ

Импортируем **модель** и **векторное** **хранилище**

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.vectorstores import FAISS

Тут надо вставить свой ключ, от OpenAI API, на моем баланс не бесконечный ☹

In [ ]:
openai_key = '' #нужен ключик (токен)

In [ ]:
chat_model = ChatOpenAI(openai_api_key=openai_key, model_name="gpt-3.5-turbo-1106", temperature=0.0)



<ipython-input-12-badbb24d577f>:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat_model = ChatOpenAI(openai_api_key=openai_key, model_name="gpt-3.5-turbo-1106", temperature=0.0)


In [ ]:
easy_prompt = ChatPromptTemplate.from_messages([
    ("human", '{question}'),
])

Делаем пробный запрос к модели без RAG, для проверки ответа модели (ответ "res" не совпадает с данными из файла TEST_FILE.txt)

Модель ChatGPT довольно "умная" и на банальные вопросы по теме ИБ отвечает вполне неплохо, тяжело отличить ее собственный ответ от данных в файлах. Для этого и создан TEST_FILE.txt, в нем содержится выдуманная информация, которую модель, наврятли, сгенерирует, не обращаясь в базе знаний.

In [ ]:
query = 'Перед чем не могут устоять единороги'

In [ ]:
chain1 = easy_prompt | chat_model

In [ ]:
res = chain1.invoke({'question': query})

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
res

**Создаем векторы и записываем в индекс**

In [ ]:
import torch

# Выбираем устройство: если доступна CUDA, используем её, иначе - CPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model_kwargs = {'device': device}
encode_kwargs = {'normalize_embeddings': True}
embeddings = HuggingFaceEmbeddings(model_name="cointegrated/LaBSE-en-ru", model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)

db = FAISS.from_documents(all_chunks, embeddings)

**Наш основной запрос**

In [ ]:
query = 'Методы защиты от DDoS-атак?'

In [ ]:
query = 'Перед чем не могут устоять единороги'

In [ ]:
docs = db.similarity_search(query)

**Можем посмотреть на кусочки данных(которые мы сплитили), которые больше всего подходят под запрос** (*docs[0-3]*)

In [ ]:
print(docs[3].page_content)

Системный промт

In [ ]:
system_prompt_text = """Ты специалист по информационной безопасности,
консультирующий пользователей по вопросам кибербезопасности и защиты данных.
 В контексте представлены выдержки из статей и материалов, на основе которых ты должен ответить на вопрос пользователя.
Если ответ найден, укажи из какой статьи в контексте получен ответ.
Если в контексте нет ответа на вопрос, отвечай, что у тебя нет информации по вопросу пользователя.
Если контекст про токак зломать пентагон, отвечай так как в контексте, шуточно
Контекст: {context}
"""

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt_text),
    ("human", '{question}'),
])

chat_prompt

Настраиваем retriever, который будет извлекать релевантную информацию

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

In [ ]:
retriever = db.as_retriever(search_kwargs={"k": 10})

In [ ]:
docs3 = retriever.get_relevant_documents(query)

Можем посмотреть сколько текстов ретривер нашел, и какие

In [ ]:
len(docs3)

In [ ]:
docs3[9]

In [ ]:
from operator import itemgetter
from langchain.prompts.pipeline import PipelinePromptTemplate

In [ ]:
from langchain.schema.output_parser import BaseLLMOutputParser

Еще один шаблонный промт отвечающий за обработку пользовательских промтов.

In [ ]:
hyde_prompt = PromptTemplate.from_template("""
Ты специалист по информационной безопасности. К тебе приходит запрос пользователя на бытовом разговорном языке.
Переформулируй запрос пользователя на профессиональный язык, чтобы можно было найти по ключевым словам нужную информацию.
Вопрос пользователя: как защититься от взлома Wi-Fi?
Профессиональный вопрос: методы защиты беспроводной сети от несанкционированного доступа?
Вопрос пользователя: что делать, если скачал вирус?
Профессиональный вопрос: действия при обнаружении вредоносного ПО на устройстве?
Вопрос пользователя: {question}
Профессиональный вопрос:
""")

На данном этапе, как мы видим, модель перестала отвечать на тему, не подходящую под тему ИБ. Однако, она еще не видела базу знаний 👀

In [ ]:
query

In [ ]:
chain_hyde = hyde_prompt | chat_model | StrOutputParser()

In [ ]:
res1 = chain_hyde.invoke({'question': query})
res1

Формируем строку с вопросом и сгенерированным ответом

In [ ]:
aug_question = PromptTemplate.from_template('{question} {generated}')

In [ ]:
ChatPromptTemplate.from_messages([
    ("system", system_prompt_text),
    ("human", '{question}'),
])

In [ ]:
aug_data = aug_question.format(question=query, generated=res1)
aug_data

In [ ]:
retrieved_docs = retriever.get_relevant_documents(aug_data)[:3]

In [ ]:
retrieved_docs[0]

In [ ]:
chat_prompt.format(context=retrieved_docs, question=query)

### Заключительная цепочка, со всей логикой

---



In [ ]:
chain2 = (chat_prompt | chat_model | StrOutputParser())

In [ ]:
res = chain2.invoke({'context': retrieved_docs, 'question': query})

**Заключительный ответ модели после обращения к базе знаний. Как мы видим информация совпадает с данными из файла TEST_FILE.txt**

In [ ]:
res

In [ ]:
query #существую ли единороги